In [1]:
import sklearn
import numpy as np
import pandas as pd
from scipy import io
import tensorflow as tf
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
import glob, os
import h5py
import sys
import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [ ]:
def _ds_parser(proto):
    featdef = {
               'data': tf.FixedLenFeature([257],dtype=tf.float32),
               'label': tf.FixedLenFeature([257],dtype=tf.float32)
              }
    example = tf.parse_single_example(proto, featdef)
    input_feature = example['data']
    clean_label = example['label']
    input_feature = tf.reshape(input_feature,(257,1,1))
    return input_feature, clean_label

In [ ]:
import glob
train_tfrecords_filenames = glob.glob('/home/ssprl2/Documents/Nikhil/DNS/tfrecord_mag/train*')
train_tfrecords_filenames = list(train_tfrecords_filenames)
print(train_tfrecords_filenames)

In [ ]:
dataset = tf.data.TFRecordDataset(train_tfrecords)
dataset = dataset.map(_ds_parser)
dataset = dataset.repeat(200)
dataset = dataset.batch(500)
iterator = dataset.make_one_shot_iterator()
X1, X2, Y = iterator.get_next()

In [2]:
#test
#mainCRNN pesq 1.59 csii 0.9 stoi 0.92

model1 = tf.keras.models.Sequential()
model1.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           input_shape = (257,1,1), filters = 257, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model1.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 129, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model1.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 65, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model1.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model1.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 33, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model1.add(tf.keras.layers.Reshape((33,33)))
model1.add(tf.keras.layers.LSTM(33, return_sequences=True,input_shape=(33, 33)))  
model1.add(tf.keras.layers.LSTM(33, return_sequences=True))  
model1.add(tf.keras.layers.Flatten())





model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           input_shape = (257,1,1), filters = 257, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model2.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 129, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model2.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 65, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model2.add(tf.keras.layers.MaxPool2D(pool_size = (2,2),  padding='same'))

model2.add(tf.keras.layers.Conv2D(kernel_size = (5,5), 
           filters = 33, activation='relu',
           kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
           bias_initializer = tf.keras.initializers.Constant(0.1), strides=(1, 1), padding = 'same' ))
model2.add(tf.keras.layers.Reshape((33,33)))
model2.add(tf.keras.layers.LSTM(33, return_sequences=True,input_shape=(33, 33)))  
model2.add(tf.keras.layers.LSTM(33, return_sequences=True))  
model2.add(tf.keras.layers.Flatten())



model_add = tf.keras.layers.Add()([model1.output,model2.output])
model_flat = tf.keras.layers.Dense(512, activation='relu',
                                   kernel_initializer= tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.05, seed=None),
                                   bias_initializer = tf.keras.initializers.Constant(0.1))(model_add)

model_final = tf.keras.layers.Dense(units = 257, activation = 'linear')(model_flat)
model = tf.keras.models.Model(inputs=[model1.input,model2.input], outputs=model_final)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 257, 1, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4_input (InputLayer)     [(None, 257, 1, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 257, 1, 257)  6682        conv2d_input[0][0]               
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 257, 1, 257)  6682        conv2d_4_input[0][0]             
______________________________________________________________________________________________

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint_name = '/home/ssprl2/Documents/Nikhil/tfrecord_keras/models/CRNN/CRNN_Weights-{epoch:03d}--{val_loss:.5f}.h5'
checkpoint = ModelCheckpoint(checkpoint_name,monitor='loss',verbose=1,save_best_only=True,mode='auto')
callback_list = [checkpoint]

In [ ]:
model.compile(optimizer=  tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
, loss='mean_absolute_error')
model.fit([X1,X1], Y, epochs=1,steps_per_epoch=100000)

In [ ]:
# # loss - mean square error, mae, Huber loss (smooth mae), log cosh, quantile 
# def mse(true, pred): 
#     return np.sum((true - pred)**2)
    
# def mae(true, pred):
#     return np.sum(np.abs(true - pred))

# # huber loss
# def huber(true, pred, delta):
#     loss = np.where(np.abs(true-pred) < delta , 0.5*((true-pred)**2), delta*np.abs(true - pred) - 0.5*(delta**2))
#     return np.sum(loss)

# # log cosh loss
# def logcosh(true, pred):
#     loss = np.log(np.cosh(pred - true))
#     return np.sum(loss)

# def tilted_loss(y,f):
#     q = 0.5
#     e = (y-f)
#     return K.mean(K.maximum(q*e, (q-1)*e), axis=-1)